# Jira Issue Wrangler

Map-1 issue: https://ubuntu-policy.atlassian.net/browse/MAP-1

Here is the Kanban Board to edit the data: https://ubuntu-policy.atlassian.net/secure/RapidBoard.jspa?rapidView=8&projectKey=MAP

In [1]:
from JiraWrangler import JiraIssueWrangler

def fetch_jira_issue(issue_id):
    issue_wrangler = JiraIssueWrangler('Map-1')
    issue = issue_wrangler.wrangle()
    
    print(issue)
    
fetch_jira_issue('Map-1')


{"Assignee": null, "Brands": [], "Components": [], "Created": "2019-10-13T09:28:52.131+0100", "Creator": "dinis.cruz", "Description": null, "Email": null, "Epic Link": null, "GDPR Article": null, "Image Url": null, "Issue Type": "Wardley Map", "Issue Links": {"MAP-2": [{"Link Type": "needs", "Direction": "Outward", "Issue Type": "Wardley Map", "Id": "10087", "Key": "MAP-2", "Status": "Product", "Summary": "Cup"}], "MAP-3": [{"Link Type": "needs", "Direction": "Outward", "Issue Type": "Wardley Map", "Id": "10088", "Key": "MAP-3", "Status": "Product", "Summary": "Tea"}], "MAP-4": [{"Link Type": "needs", "Direction": "Outward", "Issue Type": "Wardley Map", "Id": "10089", "Key": "MAP-4", "Status": "Commodity", "Summary": "Hot Water"}], "MAP-8": [{"Direction": "Inward", "Link Type": "is needed by", "Issue Type": "Wardley Map", "Id": "10093", "Key": "MAP-8", "Status": "Genesis", "Summary": "Public"}]}, "ISO27001 Standard": null, "Key": "MAP-1", "Labels": [], "Latest_Information": null, "Like